In [1]:
from bs4 import BeautifulSoup
import requests
import os
import threading

In [3]:
#scrape root directory from physionet MIMICIII waveform (only)
ROOTURL = "https://physionet.org/files/mimic3wdb/1.0"
recordsRequest = requests.get(ROOTURL + "/RECORDS-waveforms")

#convert request item in Soup obj and into vector of string names
soup = BeautifulSoup(recordsRequest.text, 'html.parser')
all_record_paths = str(soup).splitlines()[0:10]

In [4]:
idx = 0

listOfGoodBoys = []

def scrape_mimic_list(record_paths):
    for record_path in record_paths:
        scrape_mimic(record_path=record_path)

def scrape_mimic(record_path):

    record_name = record_path[3:10]

    #Will look in each record's general layout file for presence of an ABP and PLETH waveform
    #    if it exists then will download file

    RecordsInFolderRequest = requests.get(ROOTURL + "/" + record_path + "/RECORDS")
    all_records_in_folder = str(BeautifulSoup(RecordsInFolderRequest.text, 'html.parser')).splitlines()

    for record in all_records_in_folder:
        HeaderRequest = requests.get(ROOTURL + "/" + record_path + record + ".hea")
        soup_layout = BeautifulSoup(HeaderRequest.text, 'html.parser')
        signalData = str(soup_layout).splitlines()[1:]
        signalList = []
        for line in signalData:
            signalList.append(line.split()[-1])
        signal_matches = ["ABP", "PLETH"] 
        
        if not all(a in signalList for a in signal_matches):
            continue
        #use --no-parent otherwise you will download all directory files
        # For use with Windows (with WSL)
        filePath = record_path + record
        listOfGoodBoys.append(filePath)
        download_cmd = "wsl wget -r --no-parent " + ROOTURL + "/" + filePath
        # For use with Mac/Linux
        # download_cmd = "wget -r --no-parent " + ROOTURL + "/" + record_path;
        print("DOWNLOADING -- " + record_name )
        os.system(download_cmd +'.dat')
        os.system(download_cmd +'.hea')
        print("DOWNLOAD COMPLETE -- " + record_name)

#     idx = idx+1;
#     if idx>=2: return -1;

if __name__ == '__main__':
    numThreads = 8
    threadList = []

    numRecords = len(all_record_paths)//numThreads
    
    for x in range(8):
        #threadRunner = threading.Thread(target=scrape_mimic_list, args=all_record_paths[x:len(all_record_paths):numThreads])
        if x < 7:
            threadRunner = threading.Thread(target=scrape_mimic_list, args=(all_record_paths[x*numRecords:(x+1)*numRecords],))
        else:
            threadRunner = threading.Thread(target=scrape_mimic_list, args=(all_record_paths[x*numRecords:],))
        threadRunner.start()
        threadList.append(threadRunner)

    for x in range(8):
        threadList[x].join()


DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3000063
DOWNLOAD COMPLETE -- 3000063


In [13]:
folderRoot = "physionet.org/files/mimic3wdb/1.0/"
fileThreshold = 17000
for record in listOfGoodBoys:
    filePath = folderRoot+record
    fileSize =  os.path.getsize(filePath+'.dat')
    if fileSize > fileThreshold:
        # Convert to TXT
        rdsamp_cmd = 'wsl rdsamp -r ' + filePath + '-p >' + filePath + '.txt'
        print('CONVERTING TO TXT -- ' + filePath)
        os.system(rdsamp_cmd)
        print('CONVERTING TO TXT COMPLETE -- ' + filePath)
        
        # Convert to MAT
        wfdb2mat_cmd = 'wsl wfdb2mat -r ' + filePath
        print('CONVERTING TO MAT -- ' + filePath)
        os.system(wfdb2mat_cmd)
        folder = record.split('/')[0] + '/' + record.split('/')[1] + '/'
        fileName = record.split('/')[2]
        mv_cmd = 'wsl mv ' + fileName + 'm.mat ' + fileName + 'm.hea ' + folderRoot + folder 
        os.system(mv_cmd)
        print('CONVERTING TO MAT COMPLETE -- ' + filePath)
        

CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0006
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0006
CONVERTING TO MAT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0006
wsl mv 3000063_0006m.mat 3000063_0006m.hea physionet.org/files/mimic3wdb/1.0/30/3000063/
CONVERTING TO MAT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0006
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007
CONVERTING TO MAT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007
wsl mv 3000063_0007m.mat 3000063_0007m.hea physionet.org/files/mimic3wdb/1.0/30/3000063/
CONVERTING TO MAT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0007
CONVERTING TO TXT -- physionet.org/files/mimic3wdb/1.0/30/3000063/3000063_0009
CONVERTING TO TXT COMPLETE -- physionet.org/files/mimic3wdb/1.0/30/3000063/